In [ ]:
!pip install hashedindex
!apt-get update
!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-por
!pip install tika

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import hashedindex
import tika
import requests
from tika import parser
from hashedindex import textparser

# Start running the tika service
tika.initVM()

import os
import re

In [ ]:
#for dirname, _, filenames in os.walk('/kaggle/input/trec-covid-information-retrieval/CORD-19/CORD-19/document_parses/pdf_json'):
#    for filename in filenames[:5]:
#        print(os.path.join(dirname, filename))

In [ ]:
docids = pd.read_csv("../input/trec-covid-information-retrieval/docids-rnd3.txt", sep = " ", header = None)

topicColList = ["topic-id", "query"]
metadataColList = ["cord_uid", "pdf_json_files"]

topics = pd.read_csv("../input/trec-covid-information-retrieval/topics-rnd3.csv", usecols = topicColList)
metadata = pd.read_csv("../input/trec-covid-information-retrieval/CORD-19/CORD-19/metadata.csv", usecols = metadataColList)

In [ ]:
topics.head()

In [ ]:
docids.head()

In [ ]:
metadata.head()

In [ ]:
index = hashedindex.HashedIndex()

In [ ]:
metadataSize = len(metadata)
topicsSize = len(topics)

#print("Metadata Size:", metadataSize) pegar 15 mil
print("Topics Size:", topicsSize)
metadataSize = 20000

totalRuns = metadataSize * topicsSize

from IPython.display import clear_output

In [ ]:
results = []
iteration = 1
for index_t, topic in topics.iterrows():
    for index_m, data in metadata.head(metadataSize).iterrows():
        progress = (iteration / totalRuns) * 100 
        clear_output(wait=True)
        print("Progress:", progress, "(%)")
        if isinstance(data["pdf_json_files"], str):
            try:
                text = ""
                filecontent = parser.from_file("/kaggle/input/trec-covid-information-retrieval/CORD-19/CORD-19/" + data["pdf_json_files"])
                for line in filecontent["content"].splitlines():
                    if "\"text\"" in line or "\"title\"" in line:
                        text += line
                
                text = text.replace("\"text\":", "").replace("\"title\":", "").replace("\"", "")
                
                occurencesCount = text.count(topic["query"])
                
                results.append([topic["topic-id"], data["cord_uid"], occurencesCount]) 
            except:
                continue
        iteration += 1

In [ ]:
results

In [ ]:
resultDF = pd.DataFrame(results, columns = ['topic-id', 'cord-id', 'countOccur'])
resultDF = resultDF.sort_values(by = ["topic-id", "countOccur"], ascending=[True, False])

resultDF.head(10)

In [ ]:
filename = 'submission.csv'

resultDF[["topic-id","cord-id"]].to_csv(filename,index=False)

print('Saved file: ' + filename)